In [16]:
!pip install beautifulsoup4

In [17]:
!pip install requests

In [63]:
import requests
from bs4 import BeautifulSoup

def get_filmography(actor_name):
    # Prepare the actor's name for the Wikipedia URL
    formatted_name = actor_name.replace(" ", "_")
    wikipedia_url = f"https://en.wikipedia.org/wiki/{formatted_name}"

    
    response = requests.get(wikipedia_url)

    if response.status_code == 200:
         Parse the HTML content of the page
        soup = BeautifulSoup(response.text, 'html.parser')

        # Find the filmography table on the Wikipedia page
        filmography_table = None
        for table in soup.find_all("table", {"class": "wikitable"}):
            if "Filmography" in str(table):
                filmography_table = table
                break

        if filmography_table:
            filmography = []
            for row in filmography_table.find_all("tr")[1:]: 
                columns = row.find_all("td")
                if len(columns) >= 2:
                    film_title = columns[0].text.strip()
                    year = columns[1].text.strip()
                    filmography.append((film_title, year))

            # Sort the filmography in descending order by year
            filmography.sort(key=lambda x: x[1], reverse=True)

            return filmography
        else:
            return "Filmography not found on the Wikipedia page."
    else:
        return "Failed to retrieve the Wikipedia page."


actor_name = input("Enter the actor's name: ")
filmography = get_filmography(actor_name)

if isinstance(filmography, list):
    print(f"Filmography of {actor_name} in descending order:")
    for film, year in filmography:
        print(f"{year}: {film}")
else:
    print(f"Error: {filmography}")

Enter the actor's name: Salman Khan
Error: Filmography not found on the Wikipedia page.


In [68]:
import requests
from bs4 import BeautifulSoup

def get_filmography(actor_name):
    base_url = f"https://www.imdb.com/find?q={actor_name}&s=nm"
    response = requests.get(base_url)
    soup = BeautifulSoup(response.content, 'html.parser')
    #print(soup.prettify())

    # Extract the actor's IMDb ID
    results = soup.find_all("td", class_="result_text")
    if not results:
        print("Actor not found on IMDb.")
        return
    actor_page_url = results[0].find("a")["href"]

    
    actor_url = f"https://www.imdb.com{actor_page_url}filmotype=actor&sort=year,desc"

    # Send a request to the actor's filmography page
    response = requests.get(actor_url)
    soup = BeautifulSoup(response.content, 'html.parser')

    # Extract filmography data
    filmography = []
    filmography_table = soup.find("table", class_="filmography")
    if filmography_table:
        rows = filmography_table.find_all("tr")
        for row in rows[1:]: 
            columns = row.find_all("td")
            if len(columns) >= 2:
                film_title = columns[0].get_text(strip=True)
                film_year = columns[1].get_text(strip=True)
                filmography.append((film_title, film_year))

    return filmography

if __name__ == "__main__":
    actor_name = input("Enter the actor's name: ")
    filmography = get_filmography(actor_name)
    
    if filmography:
        print(f"Filmography of {actor_name} in descending order:")
        for film_title, film_year in filmography:
            print(f"{film_title} ({film_year})")
    else:
        print("No filmography data found for the actor.")

Enter the actor's name: Shah Rukh Khan
Actor not found on IMDb.
No filmography data found for the actor.
